In [1]:
import pandas as pd
import praw
import datetime as dt
from pmaw import PushshiftAPI
from IPython.display import clear_output

In [2]:
reddit = praw.Reddit(
    client_id = "sJWEgXc5LkKbsymp1BKSyA",
    client_secret = "AvxFCAQL4sChsnCUK9HfPeYUMzw3zg",
    user_agent = "testbot"
)

In [3]:
def subreddit_scrape(reddit, subreddit_name, corpus_df):
    subreddit = reddit.subreddit(subreddit_name)
    for submission in subreddit.top(limit = 100000):
        if submission.score < 1000:
            break
        corpus_df = corpus_df.append({"subreddit": subreddit_name,
                                      "title": submission.title, 
                                      "score": submission.score, 
                                      "id": submission.id,
                                      "url": submission.url},
                                     ignore_index = True)
    return corpus_df

In [4]:
api = PushshiftAPI()

def psscrape_all(reddit, subreddit_name, after_time, before_time, corpus_df):
    posts = api.search_submissions(subreddit = subreddit_name, 
                                   limit = None,
                                   after = after_time,
                                   before = before_time)
    ids = ["t3_" + post["id"] for post in posts]
    for submission in reddit.info(fullnames = ids):
        if submission.score >= 1000:
            corpus_df = corpus_df.append({"subreddit": subreddit_name,
                                          "title": submission.title, 
                                          "score": submission.score,
                                          "id": submission.id,
                                          "date": dt.datetime.fromtimestamp(submission.created).strftime('%d %B, %Y')},
                                         ignore_index = True)
            print(submission.title)
    return corpus_df

In [5]:
subreddits = ["worldnews"]
one_day = 86400
before_time = int(dt.datetime(2022,3,20,0,0).timestamp())
after_time = int(dt.datetime(2022,3,19,0,0).timestamp())
end_time = int(dt.datetime(2022,3,13,0,0).timestamp())
corpus_df = pd.DataFrame(columns = ["subreddit", "title", "score", "id", "date"])

In [6]:
while before_time > end_time:
    try:
        print(f"Currently scraping posts from {dt.datetime.fromtimestamp(after_time).strftime('%d %B, %Y')}.")
        for subreddit_name in subreddits:
            corpus_df = psscrape_all(reddit, subreddit_name, after_time, before_time, corpus_df)
        corpus_df.drop_duplicates(inplace = True, ignore_index = True)
        clear_output(wait=True)
        print(f"{dt.datetime.fromtimestamp(after_time).strftime('%d %B, %Y')} complete. Corpus contains {corpus_df.shape[0]} rows.")
        before_time -= one_day
        after_time -= one_day
    except:
        print("Error found, retrying last iteration")
        pass

12 March, 2022 complete. Corpus contains 689 rows.


In [11]:
#corpus_df.to_csv('corpus.csv', index = False)

In [7]:
corpus_df.to_csv('test_corpus.csv', index = False)